# Dissertation

## Research into the techniques and methods to achieve state of the art accuracy in flower species identification


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path

In [3]:
print(tf.__version__)

2.5.0


In [4]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [5]:
if tf.test.gpu_device_name(): 
  
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [6]:
#makes all outputs be in float format rather than exponentials
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [7]:
#Set Parameters
IMG_SIZE = 50 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'FlowerClassifierTrial.model'.format(int(time.time()))
log_dir=f'logs\\{MODEL_NAME}'
TENSORBOARD = TensorBoard(log_dir)

In [8]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Load Directory

In [9]:
URL_dir = 'E:/Github/thesis/flowerDataset/grabcut'
gan_dir = 'E:/Github/thesis/flowerDataset/new_GAN_dataset/'
GC_gan_dir = 'E:/Github/thesis/flowerDataset/grabcut_Gan_dataset/'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
loaded_images_dir = 'E:/Dissertation/data/GANloaded_images.npy'
image_train_dir ='E:/Dissertation/data/GANimage_train.npy'
label_train_dir = 'E:/Dissertation/data/GANlabel_train.npy'
image_test_dir = 'E:/Dissertation/data/GANimage_test.npy'
label_test_dir = 'E:/Dissertation/data/GANlabel_test.npy'
image_val_dir = 'E:/Dissertation/data/GANimage_val.npy'
label_val_dir = 'E:/Dissertation/data/GANlabel_val.npy'

In [10]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)
df = pd.DataFrame(data_labels)
labels = df[1][3][0] #loads the labels that is stored in dataframe format
UNIQUE_LABELS = np.unique(labels)
UNIQUE_LABELS

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102], dtype=uint8)

In [11]:
#grabcut GANs

In [12]:
grabcut_gan_dir = 'E:/Github/thesis/flowerDataset/grabcut_Gan_dataset/'

In [13]:
# #deletes normal images in the gan folder
# for folder in os.listdir(GC_gan_dir):
#         full_path = os.path.join(GC_gan_dir, folder)
#         for file in os.listdir(full_path):
#             #if ".jpg" in file:
#             os.remove(os.path.join(full_path,file))

In [36]:
def grab_cut(gan_dir): #grab cut algorithm used to created segmented flower images WARNING: very time consuming
    for folder in os.listdir(gan_dir):
        full_path = os.path.join(gan_dir, folder)
        for file in os.listdir(full_path):
            img = cv2.imread(os.path.join(full_path,file))
            name = file
            print(name)
            loc = 'E:/Github/thesis/flowerDataset/grabcut_Gan_dataset/'
            
            mask = np.zeros(img.shape[:2], np.uint8)
            bgdModel = np.zeros((1,65), np.float64)
            fgdModel = np.zeros((1,65), np.float64)
            rect = (10,5,650,650)

            cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
            mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
            img = img*mask2[:,:,np.newaxis]

            cv2.imwrite(loc+ folder +"/" + name, img)
        
    

In [37]:
#grab_cut(gan_dir)

image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_275

image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_280

image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_275

image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_280

image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_280

image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_280

image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png
image_step_27500-10.png
image_step_27500-11.png
image_step_27500-12.png
image_step_27500-13.png
image_step_27500-14.png
image_step_27500-8.png
image_step_27500-9.png
image_step_28000-0.png
image_step_28000-1.png
image_step_28000-10.png
image_step_28000-11.png
image_step_28000-12.png
image_step_28000-13.png
image_step_28000-14.png
image_step_28000-2.png
image_step_28000-3.png
image_step_28000-4.png
image_step_28000-5.png
image_step_28000-6.png
image_step_28000-8.png
image_step_28000-9.png


In [38]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        #img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
    return images

In [39]:
load_images_from_folder(URL_dir)

[array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        ...,
 
        [[  0,   0,   0],
         [  1,   1,   1],
         [ 25,  63,  56],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   1],
         [130, 165, 171],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [143, 153, 168],
         [180, 191, 207],
         ...,
         [  0,   0,   0],
  

In [40]:
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [41]:
image_data.shape

(8189, 50, 50, 3)

In [42]:
#combine image with labels
def randomised_dataset(image_data, labels):
    np.random.seed(8)
    indices = np.arange(image_data.shape[0])
    print(indices)
    np.random.shuffle(indices)
    image_data = image_data[indices]
    labels = labels[indices]
    return image_data, labels

In [43]:
image_data, labels = randomised_dataset(image_data, labels)

[   0    1    2 ... 8186 8187 8188]


In [44]:
### read the GAN grabcuts and then add it together with standard grab cut directory. DO it like the previous notebooks

In [45]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation adn testing data
def split_data(image_data, labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, labels, test_size=1 - train_ratio)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)


In [46]:
split_data(image_data,labels)

image_train (6141, 50, 50, 3)
label_train (6141,)
image_test (820, 50, 50, 3)
label_test (820,)
image_val (1228, 50, 50, 3)
label_val (1228,)


In [47]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

In [48]:
def add_gan(GC_gan_dir):
    images = []
    labels = []
    for folder in os.listdir(GC_gan_dir):
        full_path = os.path.join(GC_gan_dir, folder)
        for file in os.listdir(full_path):
            #print(os.path.join(full_path,file))
            img = cv2.resize(cv2.imread(os.path.join(full_path,file), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
            img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
            #print(img)
            if img is not None:
                images.append(img)
                index = int(folder)
                labels.append(index)
                #print(labels)
    return images, labels

In [49]:
gan_images, gan_labels = add_gan(GC_gan_dir)

In [50]:
# gan_images = np.array(GC_gan_dir)
# gan_labels = np.array(GC_gan_dir)

In [51]:
gan_images

[array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 

In [52]:
gan_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 101,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 102,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15

In [53]:
len(gan_images)

2142

In [54]:
image_train = np.concatenate((image_train, gan_images), axis=0)
label_train = np.concatenate((label_train, gan_labels),axis=0)

In [55]:
print(image_train.shape)
print(label_train.shape)

(8283, 50, 50, 3)
(8283,)


In [56]:
mlb = LabelBinarizer()
label_train = np.array(mlb.fit_transform(label_train))
label_test = np.array(mlb.fit_transform(label_test))
label_val = np.array(mlb.fit_transform(label_val))

In [57]:
image_train = image_train/255.0
image_test = image_test/255.0
image_val = image_val/255.0

In [58]:
print(image_train.shape)
print(image_test.shape)
print(image_val.shape)

(8283, 50, 50, 3)
(820, 50, 50, 3)
(1228, 50, 50, 3)


# Custom CNN model

In [59]:
batch_size = 32
epochs = 100
verbose= 2
act = 'relu' #sigmoid tanh
dropout_val = 0.4
NUM_CLASSES = 102

In [60]:
def create_cnn_model(act):
    # tf.reset_default_graph()
    model = Sequential()
    input_shape = (IMG_SIZE, IMG_SIZE, 3)
    
    # INPUT LAYER
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation(act))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    # HIDDEN LAYER 1
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))

#     #HIDDEN LAYER 2
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))
    
#     # HIDDEN LAYER 3
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))
    
    # HIDDEN LAYER 4
    model.add(Conv2D(256, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))

    # Fully Connected
    model.add(Flatten()) # converts the 3D feature maps to 1D feature vectors
#     model.add(Dense(128))
#     model.add(Activation(act))
#     model.add(Dropout(dropout_val)) # reduces overfitting

    # OUTPUT LAYER
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [61]:
model = create_cnn_model(act)

In [62]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 46, 46, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 23, 23, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 32)        0

# Evaluation

In [63]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [65]:
EXPERIMENT_NAME = "best_model"
RUN_NAME = "custom_GCGAN_1"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_param("activation function", act)
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        
        
        
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        mlflow.keras.log_model(model, RUN_NAME)        

2021/08/22 10:16:08 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Epoch 1/100


259/259 - 6s - loss: 5.1564 - accuracy: 0.0220 - val_loss: 4.8163 - val_accuracy: 0.0114
Epoch 2/100
259/259 - 2s - loss: 4.7457 - accuracy: 0.0443 - val_loss: 4.8021 - val_accuracy: 0.0179
Epoch 3/100
259/259 - 2s - loss: 4.5452 - accuracy: 0.0746 - val_loss: 4.7827 - val_accuracy: 0.0171
Epoch 4/100
259/259 - 2s - loss: 4.3632 - accuracy: 0.1076 - val_loss: 4.7609 - val_accuracy: 0.0179
Epoch 5/100
259/259 - 2s - loss: 4.2511 - accuracy: 0.1312 - val_loss: 4.7292 - val_accuracy: 0.0187
Epoch 6/100
259/259 - 2s - loss: 4.1235 - accuracy: 0.1563 - val_loss: 4.7334 - val_accuracy: 0.0204
Epoch 7/100
259/259 - 2s - loss: 4.0169 - accuracy: 0.1724 - val_loss: 4.7814 - val_accuracy: 0.0163
Epoch 8/100
259/259 - 2s - loss: 3.9321 - accuracy: 0.1911 - val_loss: 4.8255 - val_accuracy: 0.0179
Epoch 9/100
259/259 - 2s - loss: 3.8471 - accuracy: 0.2026 - val_loss: 4.7515 - val_accuracy: 0.0204
Epoch 10/100
259/259 - 2s - loss: 3.7733 - accuracy: 0.2151 - val_loss: 4.7998 - val_accuracy: 0.0179
E

Epoch 82/100
259/259 - 2s - loss: 1.8431 - accuracy: 0.5231 - val_loss: 6.0726 - val_accuracy: 0.0171
Epoch 83/100
259/259 - 2s - loss: 1.8189 - accuracy: 0.5275 - val_loss: 5.9962 - val_accuracy: 0.0163
Epoch 84/100
259/259 - 2s - loss: 1.8167 - accuracy: 0.5265 - val_loss: 6.0493 - val_accuracy: 0.0171
Epoch 85/100
259/259 - 2s - loss: 1.8237 - accuracy: 0.5271 - val_loss: 6.0834 - val_accuracy: 0.0147
Epoch 86/100
259/259 - 2s - loss: 1.8132 - accuracy: 0.5328 - val_loss: 5.9615 - val_accuracy: 0.0138
Epoch 87/100
259/259 - 2s - loss: 1.7832 - accuracy: 0.5277 - val_loss: 6.0327 - val_accuracy: 0.0147
Epoch 88/100
259/259 - 2s - loss: 1.7805 - accuracy: 0.5335 - val_loss: 6.0290 - val_accuracy: 0.0147
Epoch 89/100
259/259 - 2s - loss: 1.7888 - accuracy: 0.5331 - val_loss: 6.0864 - val_accuracy: 0.0147
Epoch 90/100
259/259 - 2s - loss: 1.7450 - accuracy: 0.5377 - val_loss: 6.0735 - val_accuracy: 0.0114
Epoch 91/100
259/259 - 2s - loss: 1.7630 - accuracy: 0.5374 - val_loss: 6.1498 - v

INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpxmw96ujl\model\data\model\assets


26/26 - 0s - loss: 6.2596 - accuracy: 0.0110
INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpm7dq3tyu\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpm7dq3tyu\model\data\model\assets


In [66]:
import winsound

winsound.Beep(1000, 1000)

In [ ]:
#function that returns loss value & metrics values
model.evaluate(image_test, label_test)

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir=logs

In [ ]:
def prediction_accuracy(image_test, model):
    #going through each of the test images
    count = 0
    for i in range(len(image_test)): #going through all test images
        img = image_test[i] #load in the image
        
        img = img.reshape(-1, 100, 100, 3) #get it in the right shape for model.predict
        
        prediction = model.predict(img).flatten() #using model predict to get what it thinks is the answer
        
        # prediction.astype(np.int) #converts predictions to integers
        
        pred_index = np.argmax(prediction) # grabs the INDEX of the best prediction
        
        max_pred = max(prediction) # grabs the SCORE of best prediction
        # print(prediction)
        # print(pred_index)
        
        test_label_value = label_test[i][pred_index] # gets the actual test label at the same index
        # print(test_label_value)
        if test_label_value == 1:
            #print("CORRECT PREDICTION")
            count += 1
            
#         # FIND THE INDEX WHERE THE VALUE = 1
#         for j in range(len(predicted_label)):
#             if predicted_label[j] == 1 : #find the point when it finds 1 
#                 if label_test[i][j] == 1: #check if at this exact point is also 1  whic means correct prediction
#                     count+=1 #increase number of correct count by 1
#                     print(count)
#                     break
    print(f"num correct - {count}")
    accuracy = (count/len(label_test))*100
    per_symbol = '%'
    print(f'accuracy is {accuracy} {per_symbol}')        

In [ ]:
image_test.shape

In [ ]:
prediction_accuracy(image_test, model)